In [4]:
import pandas as pd
from sklearn.grid_search import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
import numpy as np
from sklearn.cross_validation import cross_val_score
import matplotlib
import seaborn as sb
import statsmodels.api as sm


def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

%matplotlib inline

table_split = {'sales_prd': ['Semana','Agencia_ID','Canal_ID','Ruta_SAK','Cliente_ID','Producto_ID'],
              'sales_rtn':['Semana','Venta_uni_hoy','Venta_hoy','Dev_uni_proxima','Dev_proxima', 'Demanda_uni_equil']}


store = pd.HDFStore('../data/mystore.h5') 
model = LinearRegression()

byweek = store.select('df',column=['Producto_ID']).groupby('Semana')

byweek.apply(regress, 'Producto_ID', ['Venta_uni_hoy'])


#store.create_table_index('sales_prd', columns = ['Semana','Cliente_ID','Producto_ID','Venta_uni_hoy','Venta_hoy','Dev_uni_proxima','Dev_proxima']\
                        #,optlevel=9, kind='full')

#tempdf = pd.read_hdf('../data/mystore.h5', chunksize = 200000)
#df = pd.concat(tempdf)
#df = df.rename(columns = {'Semana':'WeekNum','Venta_uni_hoy':'SaleUnits','Venta_hoy':'SaleNum',\
#                'Dev_uni_proxima':'ReturnUnits','Dev_proxima':'ReturnNum', 'Demanda_uni_equil':'Demand',\
#                'Agencia_ID':'SalesDepotId','Canal_ID':'SalesChannelId','Ruta_SAK':'RouteId','Cliente_ID':'ClientId',\
#                'Producto_ID':'ProductId'})
#print(df.size)
#sb.barplot(data = df, x = ['WeekNum'], y=['SaleUnits'])
#sb.pairplot(x_vars = ['WeekNum'], y_vars = ['SaleUnits','ReturnUnits'], data = df, size = 8)


    #saletrain, saletest, saleLabelTrain, saleLabelTest = train_test_split(df, df['SaleUnits'], test_size = 0.4)    
    #%time print(np.sqrt(-cross_val_score(model, saletrain, saleLabelTrain, cv=10, scoring='mean_squared_error')).mean())
    #break


#saletrain, saletest, saleLabelTrain, saleLabelTest = train_test_split(df, df['SaleUnits'], test_size = 0.4)    
#%time print(np.sqrt(-cross_val_score(model, saletrain, saleLabelTrain, cv=10, scoring='mean_squared_error')).mean())

# create a store

#for iterator in pd.read_csv('../data/train.csv', iterator = True, chunksize = 200000):
#    store.append('df', iterator, data_columns = ['Semana','Cliente_ID','Producto_ID',\
#                    'Venta_uni_hoy','Venta_hoy','Dev_uni_proxima','Dev_proxima'], index = False)
#store.create_table_index('df', 
#                         columns = ['Semana','Cliente_ID','Producto_ID','Venta_uni_hoy',\
#                                        'Venta_hoy','Dev_uni_proxima','Dev_proxima'],
#                         optlevel=9, 
#                      kind='full'
#                        )



    


,Venta_uni_hoy,intercept
Semana,,
3,41.711058,20119.255195
4,38.159441,20428.053799
5,36.808297,20666.832848
6,38.468086,20352.031415
7,38.505447,20414.104260
8,38.236591,20804.610579
9,41.635268,21133.913374
